Install and Import Dependencies

In [ ]:
%pip install mediapipe opencv-python gTTS pygame

You should consider upgrading via the '/Users/joshualum/Desktop/Vscode Projects/pythonprojects/project 1: front lever/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils #drawing utilities
mp_pose = mp.solutions.pose #pose estimation model

Test Camera

In [10]:
cap = cv2.VideoCapture(0) #select the device

while cap.isOpened():
    ret, frame = cap.read() #get the frames
    if not ret:
        break
    cv2.imshow('Mediapipe Feed', frame) #popup video feed

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release() #release the device
cv2.destroyAllWindows()
cv2.waitKey(1)  

-1

Make Detections

In [79]:
import cv2
import mediapipe as mp
import pygame
from gtts import gTTS
import math
import os

# === Init ===
pygame.mixer.init()
channel = pygame.mixer.Channel(0)

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
REQUIRED = [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER,
            mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP]

def get_audio(text):
    tts = gTTS(text)
    audio_file = f"{text}.mp3"
    tts.save(audio_file)
    sound = pygame.mixer.Sound(audio_file)
    return sound, audio_file

def clean_up_audio(audio_file):
    if os.path.exists(audio_file):
        os.remove(audio_file)

def visible(landmarks):
    return all(landmarks[pt.value].visibility > 0.7 for pt in REQUIRED)

def detect_side(landmarks):
    ls, rs, lh, rh = [landmarks[pt.value] for pt in REQUIRED]
    x_vals = [ls.x, rs.x, lh.x, rh.x]
    if max(x_vals) - min(x_vals) < 0.03:
        return "Right" if (ls.z + lh.z) / 2 > (rs.z + rh.z) / 2 else "Left"
    return

def calculate_angle(a, b, c):
    ba = [a[0] - b[0], a[1] - b[1]]
    bc = [c[0] - b[0], c[1] - b[1]]
    dot = ba[0] * bc[0] + ba[1] * bc[1]
    mag_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    mag_bc = math.sqrt(bc[0]**2 + bc[1]**2)
    if mag_ba == 0 or mag_bc == 0:
        return 0
    cos_angle = dot / (mag_ba * mag_bc)
    angle = math.acos(max(min(cos_angle, 1), -1))
    return math.degrees(angle)

# === Webcam Loop ===
cap = cv2.VideoCapture(0)
last_spoken = None
side_locked = None
side_frame_counter = 0
reset_counter = 0
RESET_THRESHOLD = 50

with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.6) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if not (results.pose_landmarks and visible(results.pose_landmarks.landmark)):
            side_locked = None
            side_frame_counter = 0
            reset_counter = 0
            cv2.putText(image, "Move back: shoulders and hips must be visible", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
        else:
            lm = results.pose_landmarks.landmark
            side = detect_side(lm)

            if side in ["Left", "Right"]:
                if side_locked != side:
                    side_frame_counter += 1
                    if side_frame_counter > 10:
                        side_locked = side
                        reset_counter = 0
                        if last_spoken != side_locked:
                            channel.stop()
                            sound, audio_file = get_audio(side_locked)
                            channel.play(sound)
                            last_spoken = side_locked
                            clean_up_audio(audio_file)
                else:
                    reset_counter = 0
            else:
                side_frame_counter = 0
                reset_counter += 1
                if reset_counter > RESET_THRESHOLD:
                    side_locked = None
                    last_spoken = None

            # Message 1: Turn Prompt
            if not side_locked:
                cv2.putText(image, "Please turn to the side", (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)

            # Message 2: View status
            view_color = (0, 255, 0) if side_locked else (0, 255, 255)
            view_text = f"View: {side_locked or 'Detecting...'}"
            cv2.putText(image, view_text, (10, 130),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, view_color, 4)

            # Message 3: Shoulder Angle
            angle = None
            if side_locked == "Left":
                elbow = lm[mp_pose.PoseLandmark.LEFT_ELBOW.value]
                shoulder = lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                hip = lm[mp_pose.PoseLandmark.LEFT_HIP.value]
            elif side_locked == "Right":
                elbow = lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
                shoulder = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                hip = lm[mp_pose.PoseLandmark.RIGHT_HIP.value]

            if side_locked in ["Left", "Right"]:
                angle = calculate_angle(
                    (elbow.x, elbow.y), (shoulder.x, shoulder.y), (hip.x, hip.y)
                )

            if angle:
                cv2.putText(image, f"Shoulder Angle: {int(angle)} deg", (10, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 4)

            # === Diagnostic Info ===
            if results.pose_landmarks:
                diag_names = ["L_SHOULDER", "R_SHOULDER", "L_HIP", "R_HIP", "L_EAR", "R_EAR", "L_ELBOW", "R_ELBOW"]
                diag_ids = [mp_pose.PoseLandmark.LEFT_SHOULDER.value,
                            mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
                            mp_pose.PoseLandmark.LEFT_HIP.value,
                            mp_pose.PoseLandmark.RIGHT_HIP.value,
                            mp_pose.PoseLandmark.LEFT_EAR.value,
                            mp_pose.PoseLandmark.RIGHT_EAR.value,
                            mp_pose.PoseLandmark.LEFT_ELBOW.value,
                            mp_pose.PoseLandmark.RIGHT_ELBOW.value]
                for i, (idx, name) in enumerate(zip(diag_ids, diag_names)):
                    lm_i = results.pose_landmarks.landmark[idx]
                    info = f"{name}: x={lm_i.x:.2f}, y={lm_i.y:.2f}, z={lm_i.z:.2f}, v={lm_i.visibility:.2f}"
                    y_pos = 270 + 60 * i
                    cv2.putText(image, info, (10, y_pos),
                                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Show frame
        cv2.imshow("Pose Debug", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)
pygame.mixer.quit()

I0000 00:00:1747064253.790072  298418 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
W0000 00:00:1747064253.871308  585432 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747064253.882459  585432 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
